In [4]:
import numpy as np
import pandas as pd 

Transform to csv files

In [78]:
import pandas as pd 
tsv_file="/Users/gregorsink/Desktop/PR Projekt/Data/title.akas.tsv"
csv_table=pd.read_table(tsv_file,sep='\t')
csv_table.to_csv('/Users/gregorsink/Desktop/PR Projekt/Data/Dataakas.csv',index=False)

Preberimo datoteke

In [23]:
import pandas as pd
from datetime import datetime
movies = pd.read_csv("/Users/gregorsink/Desktop/PR Projekt/Data/Dataakas.csv")
ratings = pd.read_csv("/Users/gregorsink/Desktop/PR Projekt/Data/Dataratings.csv")
netflix = pd.read_csv("/Users/gregorsink/Desktop/PR Projekt/Data/netflix_titles.csv")
#crew = pd.read_csv("/Users/gregorsink/Desktop/PR Projekt/Data/Datacrew.csv")
#names = pd.read_csv("/Users/gregorsink/Desktop/PR Projekt/Data/Datanames.csv")


In [29]:
ratings = ratings.rename(columns = {'tconst':'titleId'})


In [30]:
originals = movies[movies["types"] == "original"]
#originals = originals[["titleId", "title"]]

originals = originals.merge(ratings, on="titleId")
#netflix1 = netflix.merge(originals, on="title")

In [32]:
def najdi(table):
    if table.empty:
        return table
    else:
        return table.iloc[table["numVotes"].argmax()]

In [33]:
i = 0
netflix1 = netflix
for idx, naslov in enumerate(netflix1["title"]):
    i += 1
    table = originals[originals["title"] == naslov]
    if table.empty:
        pass
    else:
        a = najdi(originals[originals["title"] == naslov])
        netflix1.loc[idx, "averageRating"] = a["averageRating"]
        netflix1.loc[idx, "numVotes"] = a["numVotes"]

    if i%100 == 0:
        print(i)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200


In [38]:
netflix1

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,averageRating,numVotes
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...,3.1,249.0
1,80117401,Movie,Jandino: Whatever it Takes,0,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...,5.2,18.0
2,70234439,TV Show,Transformers Prime,0,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob...",7.8,4968.0
3,80058654,TV Show,Transformers: Robots in Disguise,0,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...,6.0,774.0
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6229,80000063,TV Show,Red vs. Blue,0,"Burnie Burns, Jason Saldaña, Gustavo Sorola, G...",United States,0,2015,NR,13 Seasons,"TV Action & Adventure, TV Comedies, TV Sci-Fi ...","This parody of first-person shooter games, mil...",8.3,8711.0
6230,70286564,TV Show,Maron,0,"Marc Maron, Judd Hirsch, Josh Brener, Nora Zeh...",United States,0,2016,TV-MA,4 Seasons,TV Comedies,"Marc Maron stars as Marc Maron, who interviews...",7.7,4423.0
6231,80116008,Movie,Little Baby Bum: Nursery Rhyme Friends,0,0,0,0,2016,0,60 min,Movies,Nursery rhymes and original music for children...,0.0,0.0
6232,70281022,TV Show,A Young Doctor's Notebook and Other Stories,0,"Daniel Radcliffe, Jon Hamm, Adam Godley, Chris...",United Kingdom,0,2013,TV-MA,2 Seasons,"British TV Shows, TV Comedies, TV Dramas","Set during the Russian Revolution, this comic ...",0.0,0.0


In [49]:
from imdb import IMDb

movs = IMDb()

movie = movs.get_movie("8672310")
movie['rating']

KeyError: 'rating'

In [5]:
netflix1 = pd.read_csv("/Users/gregorsink/Desktop/PR Projekt/PR19KKZDBMPBPGS/Netflix_all.csv")

In [15]:
from imdb import IMDb
from imdb import IMDbDataAccessError

movs = IMDb()
i = 0

for idx, naslov in enumerate(netflix1["title"]):
    i += 1
    if netflix1.loc[idx, "director"] == "0":
        m = movs.search_movie(naslov)
        if len(m) > 0:
            try:
                data = movs.get_movie(m[0].movieID)
                netflix1.loc[idx, "director"] = data["director"][0]['name']
            except KeyError as e:
                netflix1.loc[idx, "director"] = "-1"
            except (URLError, IMDbDataAccessErro) as e:
                print(data)
    if i%100 == 0:
        print(i)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200


In [23]:
netflix1.to_csv("/Users/gregorsink/Desktop/PR Projekt/PR19KKZDBMPBPGS/Netflix_all.csv", index=False)

In [22]:
netflix1

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,averageRating,numVotes
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...,3.1,249.0
1,80117401,Movie,Jandino: Whatever it Takes,-1,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...,5.2,18.0
2,70234439,TV Show,Transformers Prime,-1,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob...",7.8,4968.0
3,80058654,TV Show,Transformers: Robots in Disguise,-1,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...,6.0,774.0
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...,5.2,4980.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6229,80000063,TV Show,Red vs. Blue,-1,"Burnie Burns, Jason Saldaña, Gustavo Sorola, G...",United States,0,2015,NR,13 Seasons,"TV Action & Adventure, TV Comedies, TV Sci-Fi ...","This parody of first-person shooter games, mil...",8.3,8711.0
6230,70286564,TV Show,Maron,-1,"Marc Maron, Judd Hirsch, Josh Brener, Nora Zeh...",United States,0,2016,TV-MA,4 Seasons,TV Comedies,"Marc Maron stars as Marc Maron, who interviews...",7.7,4423.0
6231,80116008,Movie,Little Baby Bum: Nursery Rhyme Friends,-1,0,0,0,2016,0,60 min,Movies,Nursery rhymes and original music for children...,5.0,6.0
6232,70281022,TV Show,A Young Doctor's Notebook and Other Stories,-1,"Daniel Radcliffe, Jon Hamm, Adam Godley, Chris...",United Kingdom,0,2013,TV-MA,2 Seasons,"British TV Shows, TV Comedies, TV Dramas","Set during the Russian Revolution, this comic ...",7.9,13839.0
